# Laboratorio 7: Sistema de Recomendaciones #

### Importación de librerías ###

In [1]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import json

### Funciones Predeterminadas ###

### Descarga de la data y preparación ###

Se carga el dataset, se le modifica los nombres a las columnas para que estén los correctos y se observa cómo se distribuye inicialmente la data.

In [3]:
metadata = pd.read_csv('movies_metadata.csv')

links = pd.read_csv('links_small.csv')

ratings = pd.read_csv('ratings_small.csv')

C:\Users\GUILLERMO\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Verificaremos cómo están formadas las tablas y los tipos de datos que tienen.

In [4]:
metadata.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [5]:
metadata.dtypes

adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
dtype: object

In [6]:
metadata.isna().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [7]:
links.head(5)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [8]:
links.dtypes

movieId      int64
imdbId       int64
tmdbId     float64
dtype: object

In [9]:
links.isna().sum()

movieId     0
imdbId      0
tmdbId     13
dtype: int64

In [10]:
ratings.head(5)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [11]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [12]:
ratings.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

Se cambiará el nombre de ciertas columnas del dataset 'metadata' para que coincidan con las otras. También se eliminará la columna 'timestamp' ya que se considera que no ayudarán en el resto del proceso.

In [13]:
metadata = metadata.rename(columns={'imdb_id': 'tmdbId', 'id':'imdbId'})
ratings = ratings.drop('timestamp', axis = 1)

In [14]:
metadata.head(3)

,adult,belongs_to_collection,budget,genres,homepage,imdbId,tmdbId,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [15]:
ratings.head(5)

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [16]:
ratings.dtypes

userId       int64
movieId      int64
rating     float64
dtype: object

No podemos eliminar los datos faltantes y si se dejan como NaN es probable que nos encontremos con problemas en el futuro, por lo que se considera que la mejor opción es convertirlos en "0" para dejar la data completa y no tener problemas. (Este paso se hará más adelante)

### Proceso de JSON ###

In [17]:
metadata['genres'] = metadata['genres'].str.replace("\'","\"")
#Creamos una lista que reciba los datos de 'geners'
columna_json = ['genres']

Funciones de JSON

In [18]:
def funcion_json(x):
    return json.loads(x)

for x in columna_json:
    metadata[x] = metadata[x].apply(funcion_json)

In [19]:
def funcion_json_2(x):
    w = []
    for y in range(len(x[0])):
        for k, v in x[0][y].items():
            w.append(str(k)+"_"+str(v))
    for z in range(len(w)):
        if w[z] not in metadata.columns:
            metadata[w[z]] = 0
            metadata[w[z]].iloc[x.name] = 1
        else:
            metadata[w[z]].iloc[x.name] = 1
        return
for x in columna_json:
    metadata[[x]].apply(funcion_json_2, axis = 1)

C:\Users\GUILLERMO\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [20]:
metadata.head(3)

,adult,belongs_to_collection,budget,genres,homepage,imdbId,tmdbId,original_language,original_title,overview,...,id_27,id_99,id_9648,id_37,id_10770,id_10752,id_10769,name_Carousel Productions,name_Aniplex,name_Odyssey Media
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,0,0,0,0,0,0,0,0,0,0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,0,0,0,0,0,0,0,0,0,0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0,0,0,0,0,0,0,0,0,0


In [21]:
metadata.dtypes

adult                         object
belongs_to_collection         object
budget                        object
genres                        object
homepage                      object
imdbId                        object
tmdbId                        object
original_language             object
original_title                object
overview                      object
popularity                    object
poster_path                   object
production_companies          object
production_countries          object
release_date                  object
revenue                      float64
runtime                      float64
spoken_languages              object
status                        object
tagline                       object
title                         object
video                         object
vote_average                 float64
vote_count                   float64
id_16                          int64
id_12                          int64
id_10749                       int64
i

In [22]:
dataset = pd.merge(metadata, links, on = 'tmdbId')

ValueError: You are trying to merge on object and float64 columns. If you wish to proceed you should use pd.concat

Ahora debemos arreglar el formato de la data para poder unirla (el cambio de nombre de antes ayudará a esto)

In [23]:
links['tmdbId'] = links['tmdbId'].astype(object)
dataset = pd.merge(metadata, links, on = 'tmdbId')
dataset = pd.merge(dataset, ratings, on = 'movieId')
dataset.head(2)

,adult,belongs_to_collection,budget,genres,homepage,imdbId_x,tmdbId,original_language,original_title,overview,...,id_9648,id_37,id_10770,id_10752,id_10769,name_Carousel Productions,name_Aniplex,name_Odyssey Media,movieId,imdbId_y
0,False,NaN,1000000,"[{'id': 80, 'name': 'Crime'}]",NaN,36337,NaN,en,Delusion,"In this fast-paced, noirish road movie, a comp...",...,0,0,0,0,0,0,0,0,720,118114
1,False,NaN,1000000,"[{'id': 80, 'name': 'Crime'}]",NaN,36337,NaN,en,Delusion,"In this fast-paced, noirish road movie, a comp...",...,0,0,0,0,0,0,0,0,721,114103


Ahora llevaremos a cabo el proceso de la conversión de NaN's a 0s

In [26]:
dataset = dataset.fillna(0)